# Notebook 5.2 语音识别

语音识别，又称自动语音识别（ASR），是一种将口语转换为书面格式或根据口头命令执行特定操作的技术。它涉及分析语音模式、语音学和语言结构的机器学习模型，以准确地转录和理解人类语音。

[Whisper](https://openai.com/research/whisper) 是一个由 OpenAI 发布的流行的开源模型，用于 ASR 和语音翻译。Whisper 能够转录多种语言的语音，并将这些语言翻译成英语。

由于 Whisper 的底层是基于 Transformer 的编码器-解码器架构，因此可以使用 BigDL-LLM INT4 优化功能对其进行有效优化。在本教程中，我们将指导您在 BigDL-LLM 优化的 Whisper 模型上构建一个语音识别应用程序，该应用程序可以将音频文件转录/翻译为文本。

## 5.2.1 安装程序包

如果您还没有设置环境，请先按照 [第二章](../ch_2_Environment_Setup/README.md) 中的说明进行设置。然后安装 bigdl-llm：

In [ ]:
!pip install bigdl-llm[all]

由于需要处理音频文件，您还需要安装用于音频分析的 `librosa` 软件包。

In [ ]:
!pip install -U librosa

## 5.2.2 下载音频文件

首先，让我们准备一些音频文件。例如，您可以从[Blizzard 2013](https://www.cstr.ed.ac.uk/projects/blizzard/)数据集中下载一个[英文音频文件样本](https://audio-samples.github.io/samples/mp3/blizzard_tts_unbiased/sample-0/real.mp3)，以及从[AIShell](https://www.aishelltech.com/kysjcp)数据集中下载一个[中文音频文件样本](https://github.com/chenmingxiang110/Chinese-automatic-speech-recognition/raw/master/data/test.wav)。这里我们随机选择了一个英文音频文件和一个中文音频文件。您可以根据自己的喜好选择不同的音频文件。

您可以播放下载完成的音频：

In [ ]:
import IPython

IPython.display.display(IPython.display.Audio("audio_en.mp3"))
IPython.display.display(IPython.display.Audio("audio_zh.mp3"))

## 5.2.3 加载预训练好的 Whisper 模型

现在，让我们加载一个经过预训练的 Whisper 模型，例如 [whisper-medium](https://huggingface.co/openai/whisper-medium) 。OpenAI 发布了各种尺寸的预训练 Whisper 模型（包括 [whisper-small](https://huggingface.co/openai/whisper-small)、[whisper-tiny](https://huggingface.co/openai/whisper-tiny) 等），您可以选择最符合您要求的模型。

您只需在 `bigdl-llm` 中使用单行 `transformers`-style API，即可加载具有 INT4 优化功能的 `whisper-medium`（通过指定 `load_in_4bit=True`），如下所示。请注意，对于 Whisper，我们使用了 `AutoModelForSpeechSeq2Seq` 类。

In [ ]:
from bigdl.llm.transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained(pretrained_model_name_or_path="openai/whisper-medium",
                                                  load_in_4bit=True)

## 5.2.4 加载 Whisper Processor

无论是音频预处理还是将模型输出从标记转换为文本的后处理，我们都需要 Whisper Processor。您只需使用官方的 `transformers` API 加载 `WhisperProcessor` 即可：

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(pretrained_model_name_or_path="openai/whisper-medium")

## 5.2.5 转录英文音频

使用带有 INT4 优化功能的 BigDL-LLM 优化 Whisper 模型并加载 Whisper Processor 后，就可以开始通过模型推理转录音频了。

让我们从英语音频文件 `audio_en.mp3` 开始。在将其输入 Whisper Processor 之前，我们需要从原始语音波形中提取序列数据：

In [ ]:
import librosa

data_en, sample_rate_en = librosa.load("audio_en.mp3", sr=16000)

> **注意**
>
> 对于 `whisper-medium`，其 `WhisperFeatureExtractor`（`WhisperProcessor` 的一部分）默认使用 16,000Hz 采样率从音频中提取特征。关键的是要用模型的 `WhisperFeatureExtractor` 以采样率加载音频文件，以便精确识别。

然后，我们就可以根据序列数据转录音频文件，使用的方法与使用官方的 `transformers` API 完全相同：

In [6]:
import torch
import time

# 定义任务类型
forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

with torch.inference_mode():
    # 为 Whisper 模型提取输入特征
    input_features = processor(data_en, sampling_rate=sample_rate_en, return_tensors="pt").input_features

    # 为转录预测 token id
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()

    # 将 token id 解码为文本
    transcribe_str = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'English Transcription', '-'*20)
    print(transcribe_str)

Inference time: xxxx s
-------------------- English Transcription --------------------
[' Book me a reservation for mid-day at French Camp Academy.']


> **注意**
>
> `forced_decoder_ids` 为不同语言和任务（转录或翻译）定义上下文 token 。如果设置为 `None`，Whisper 将自动预测它们。


## 5.2.6 转录中文音频并翻译成英文

现在把目光转向中文音频 `audio_zh.mp3`。Whisper 可以转录多语言音频，并将其翻译成英文。这里唯一的区别是通过 `forced_decoder_ids` 来定义特定的上下文 token：

In [7]:
# 提取序列数据
data_zh, sample_rate_zh = librosa.load("audio_zh.mp3", sr=16000)

# 定义中文转录任务
forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task="transcribe")

with torch.inference_mode():
    input_features = processor(data_zh, sampling_rate=sample_rate_zh, return_tensors="pt").input_features
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()
    transcribe_str = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'Chinese Transcription', '-'*20)
    print(transcribe_str)

# 定义中文转录以及翻译任务
forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task="translate")

with torch.inference_mode():
    input_features = processor(data_zh, sampling_rate=sample_rate_zh, return_tensors="pt").input_features
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()
    translate_str = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'Chinese to English Translation', '-'*20)
    print(translate_str)

Inference time: xxxx s
-------------------- Chinese Transcription --------------------
['制作时将各原料研磨']
Inference time: xxxx s
-------------------- Chinese to English Translation --------------------
[' When making the dough, grind the ingredients.']


## 5.2.7 后续学习

在接下来的章节中，我们将探讨如何将 BigDL-LLM 与 langchain 结合使用，langchain 是一个专为使用语言模型开发应用程序而设计的框架。有了 langchain 集成，应用程序开发过程就可以变得简单。